In [6]:
# 必要なインポートを実施
from bs4 import BeautifulSoup
import requests
import pandas as pd
from pandas import Series,DataFrame
from datetime import datetime as dt
import re

In [4]:
# 競馬情報の取得をWebスクレイピングで実施

# URLからコンテンツを取得する
def url_to_soup(url):
    req = requests.get(url)
    return BeautifulSoup(req.content, 'html.parser')

# 各馬の過去１０レースリンクを取得
def horse_page_link(url):
    soup = url_to_soup(url)
    link_list = [HOME_URL + x.get('href') for x in soup.find_all('a', class_='tx-mid tx-low') ]
    return link_list

In [30]:
p = re.compile("")
tag_to_text = lambda x: p.sub("", x).split('\n') 
split_tr = lambda x: str(x).split('</tr>')

def get_previous_race_row(soup):
    race_table = soup.select("table.tb01")[2]
    print(race_table)
    return [tag_to_text(x)  for x in split_tr(race_table)]

def horse_data(url, race_date):
    soup = url_to_soup(url)

    # 過去のレースデータ
    pre_race_data = get_previous_race_row(soup)
    print(pre_race_data)
    df = pd.DataFrame(pre_race_data)[1:][[2,3,10,11,13,14,15,19,23]].dropna().rename(columns={
        2:'date', 3:'place', 10:'len', 11:'wether', 13:'popularity', 14:'rank', 15:'time',19:'weight',23:'money'})
    return df

def result_data(url):
    soup = url_to_soup(url)

    # 土の状態
    condition = soup.find(id="race-data02").get_text().replace('\n','').split(';')[1].split('　')[2][0:2]

    # レースの長さ
    race_len = int(soup.find(id="race-data01-a").get_text().replace('\n','').split('　')[3].replace(',','')[1:4])

    # 1位の馬番
    p = re.compile('<td class="al-center">')
    hukusyo_list = []
    hukusyo_list.append(int(p.sub("", str(soup.find_all('tr', class_='bg-1chaku')[0]).split('</td>')[2]).replace('\n','') ))

    # レース日
    race_date_str = soup.find(id="race-data01-a").get_text().replace('\n','').split(';')[0].split('日')[0]
    race_date = dt.strptime(race_date_str, '%Y年%m月%d')
    return hukusyo_list, condition, race_len, race_date

a, b, c, d = result_data('https://www.nankankeiba.com/result/2018082321060403.do')
print(d)
df = horse_data('https://www.nankankeiba.com/result/2018082321060403.do', d)
df = add_race_data(df)

2018-08-23 00:00:00
<table class="tb01 w100pr stripe bg-over" summary="コーナー通過順">
<tr>
<th class="al-left" colspan="2" scope="row">コーナー通過順</th>
</tr>
<tr>
<td class="al-center">向正面</td>
<td>8,(10,12),6,(1,4,11),5,2,7,9,3</td>
</tr>
<tr>
<td class="al-center">３角</td>
<td>8,12,10,6,11,4,1,5,9,(2,7),3</td>
</tr>
<tr>
<td class="al-center">４角</td>
<td>8,12,(10,6),(4,11),(1,5),9,2,(7,3)</td>
</tr>
</table>
[['<table class="tb01 w100pr stripe bg-over" summary="コーナー通過順">', '<tr>', '<th class="al-left" colspan="2" scope="row">コーナー通過順</th>', ''], ['', '<tr>', '<td class="al-center">向正面</td>', '<td>8,(10,12),6,(1,4,11),5,2,7,9,3</td>', ''], ['', '<tr>', '<td class="al-center">３角</td>', '<td>8,12,10,6,11,4,1,5,9,(2,7),3</td>', ''], ['', '<tr>', '<td class="al-center">４角</td>', '<td>8,12,(10,6),(4,11),(1,5),9,2,(7,3)</td>', ''], ['', '</table>']]


KeyError: '[10 11 13 14 15 19 23] not in index'

In [29]:
# 馬場状態のカラム内容を文字列によって変更する
def add_soil_columns(row):
        row['soil_heavy'] = 1 if row['wether'][-2:] =='/重'  else 0
        row['soil_s_heavy'] = 1 if row['wether'][-2:] =='稍重'  else 0
        row['soil_good'] = 1 if row['wether'][-2:] =='/良'  else 0
        row['soil_bad'] = 1 if row['wether'][-2:] =='不良'  else 0
        return row

# レースデータのカラムを加工
def add_race_data(df):
    df_ =pd.DataFrame()
    for idx, row in df.iterrows():
        if row['popularity'] == '':
            continue

        # 馬場状態
        row = add_soil_columns(row)

        row['money']=int(row['money'].replace(',','')) 
        row['horse_cnt'] = int(row['rank'].split('/')[1])
        row['result_rank'] = int(row['rank'].split('/')[0])
        row['len'] = int(row['len'][0:4])
        row['popularity'] = int(row['popularity'])
        row['weight'] = int(row['weight'])

        # 　競馬場の一致
        row['same_place'] = 1 if row['place'].startswith(PLACE)  else 0

        # タイム(秒)
        try:
            time = datetime.datetime.strptime(row['time'], '%M:%S.%f')
            row['sec'] = time.minute*60 + time.second + time.microsecond/1000000 
        except ValueError:
            time = datetime.datetime.strptime(row['time'], '%S.%f')
            row['sec'] = time.second + time.microsecond/1000000

        row['sec'] = int(row['sec']) 

        df_ = df_.append(row, ignore_index=True)
    return df_